In [35]:
import re
import json
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

In [73]:
def except_zero(value: str | int):
    try:
        var = int(NUM_REGEX.sub('', str(value)))
    except:
        var = 0
        
    return var

def squeeze_data(row: dict, search_word: str) -> dict:
    template = {
        'id': '.'.join(row['url'].split('/')[-2:]),
        'search_word': search_word,
        'title': row['title'],
        'url': row['url'],
        'blog_name': row['blog_name'],
        'date': row['date']
    }
    
    template['content'] = '\n\n'.join([c['content'] for c in row['contents'] if c['info'] == 'text'])
    template['content_len'] = len(template['content'])
    
    template['content_hash_cnt'] = len(HASH_REGEX.findall(template['content']))
    template['article_hash_cnt'] = except_zero(row['additional']['hash_tags'])

    template['like_cnt'] = except_zero(row['additional']['love_cnt'])
    template['emoticon_cnt'] = len([c for c in row['contents'] if (c['info'] == 'img') and (c['content'].startswith('https://storep-phinf.pstatic.net'))])
    template['total_post'] = except_zero(row['additional']['article_cnt'])
    template['adpost_yn'] = except_zero(row['additional']['ad_post'])
    template['link_cnt'] = len([c for c in row['contents'] if c['info'] == 'link'])
    template['map_yn'] = len([c for c in row['contents'] if c['info'] == 'map'])
    template['video_yn'] = except_zero(row['additional']['video'])
    
    template['phone_yn'] = len(PHONE_REGEX.findall(template['content']))
    
    return template

In [74]:
HASH_REGEX = re.compile('#[가-힣a-zA-Z0-9]+')
NUM_REGEX = re.compile('[^0-9]')
PHONE_REGEX = re.compile('[0-9]{2,4}[-\.][0-9]{3,4}[-\.][0-9]{4}')

In [75]:
parsed_list = glob("./data/semi-structured/*.json")
parsed_list

['./data/semi-structured/홍대 회식 맛집.json',
 './data/semi-structured/홍대 데이트 맛집.json',
 './data/semi-structured/성수 데이트 맛집.json',
 './data/semi-structured/압구정 회식 맛집.json',
 './data/semi-structured/강남역 데이트 맛집.json',
 './data/semi-structured/영등포 회식 맛집.json',
 './data/semi-structured/성수 회식 맛집.json',
 './data/semi-structured/압구정 데이트 맛집.json',
 './data/semi-structured/강남역 회식 맛집.json',
 './data/semi-structured/범계 회식 맛집.json',
 './data/semi-structured/범계 데이트 맛집.json',
 './data/semi-structured/영등포 데이트 맛집.json']

In [76]:
df = []
for path in tqdm(parsed_list):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
    fn = path.split('/')[-1].split('.')[0]
    df.append([squeeze_data(d, fn) for d in data])
    

100%|██████████| 12/12 [00:00<00:00, 13.16it/s]


In [77]:
df = sum(df, [])

In [78]:
df = pd.DataFrame(df)
df

,id,search_word,title,url,blog_name,date,content,content_len,content_hash_cnt,article_hash_cnt,like_cnt,emoticon_cnt,total_post,adpost_yn,link_cnt,map_yn,video_yn,phone_yn
0,shuenmama.223027650182,홍대 회식 맛집,<b>홍대</b> 고기집 데이트 쟁반한상 삼겹살 <b>맛집</b> <b>회식</b>...,https://blog.naver.com/shuenmama/223027650182,shuen,20230225,홍대에서 데이트 하기로 한 주말\n진짜 간만에 홍대\n전에 항상 세우던 공영주차장이...,3094,10,12,0,0,523,3,0,1,1,0
1,rosepink1974.223153722255,홍대 회식 맛집,<b>홍대</b>삼겹살 청년화로 1987 이베리코 연남동고기집 <b>회식맛집</b>,https://blog.naver.com/rosepink1974/223153722255,예쁜 달코미의 단맛 인생,20230712,청년화로1987\n서울 마포구 동교로 219 1층\n청년화로 1987\n홍대입구역 ...,2345,9,9,15,1,418,3,0,1,1,0
2,mou25.223209216526,홍대 회식 맛집,"합정 <b>맛집</b> <b>홍대 회식</b>장소로 딱, 느낌 있는 소고기 고깃집...",https://blog.naver.com/mou25/223209216526,생애 기록장,20230912,"매번 느끼는 거지만,\n회식장소 하나는\n기가 막히게 섭외하는 울 주임님.\n\n얼...",2904,12,12,0,0,554,3,0,1,0,0
3,lulu_l.223118434610,홍대 회식 맛집,<b>홍대</b> <b>맛집</b> 합정 갈비가 부드러운 소고기집 연막탄 <b>회식...,https://blog.naver.com/lulu_l/223118434610,안나의 일상공유,20230602,홍대 소고기 맛집\n연막탄\n\n남자친구가 맛있는 고깃집을 알고\n있다길해 합정 맛...,2810,0,7,9,0,222,3,0,2,0,0
4,ruston_.223161590597,홍대 회식 맛집,"<b>홍대회식</b>, 육즙 폭발하는 소고기 <b>맛집</b> '일편등심 <b>홍대...",https://blog.naver.com/ruston_/223161590597,로빈이 토끼란 사실을 알고있었나?,20230720,안녕하세요. LoLCake입니다.\n\n\n최근 이직을 준비하는 동료의 축하 파티를...,2299,2,2,0,1,573,3,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11077,suitbull.222112955908,영등포 데이트 맛집,"<b>영등포</b> 돈까스 <b>맛집</b>, 타임스퀘어 <b>데이트</b> ::간코::",https://blog.naver.com/suitbull/222112955908,SUITBULL(숯불)의 미식 그리고 풍류,20201011,안녕하세요.\n\n미식을 즐기는\n악식가(樂食家)\nSUITBULL (숯불) 입니다...,1642,0,11,14,0,231,3,0,2,1,1
11078,dusal0652.222413131913,영등포 데이트 맛집,[<b>영등포</b> 문래동] 문래창작촌에서도 유명한 양키통닭,https://blog.naver.com/dusal0652/222413131913,기록하는 습관,20210628,날씨가 좋은 어느 날\n문래에서 일어난 일ㅎㅎㅎㅎㅎ\n\n친구랑 같이 가기로 한 '...,1996,12,12,6,0,171,3,0,2,1,0
11079,shinesparis.221509975993,영등포 데이트 맛집,<b>영등포</b>역 <b>맛집</b> '럭치미' 인도요리 커리 완전맛있어~!,https://blog.naver.com/shinesparis/221509975993,귀여운 서소남매,20190410,인도요리를 좋아하는편이라 신랑이랑 가끔 먹으러가곤한다.\n영등포역에 위치한 인도요리...,2350,0,11,5,0,911,3,0,2,0,0
11080,sleeping_yulmu.223070079287,영등포 데이트 맛집,[서울/여의도] 블링 블링 빛이 나는 맛 좋은 삼겹살! '돈블랑...,https://blog.naver.com/sleeping_yulmu/22307007...,sleeping_day,20230410,예년보다 벚꽃이 일찍 개화한 올해!\n친구들과 꽃놀이는 빼놓을 수가 없죠.\n함께 ...,1745,40,30,0,0,257,3,0,1,0,0


In [83]:
df = df[df.content_len > 0].drop_duplicates('id').reset_index(drop=True)

In [87]:
df.to_csv('./data/preprocessed_data.csv', encoding='utf-8')

In [89]:

pd.read_csv('./data/preprocessed_data.csv', index_col=0)

,id,search_word,title,url,blog_name,date,content,content_len,content_hash_cnt,article_hash_cnt,like_cnt,emoticon_cnt,total_post,adpost_yn,link_cnt,map_yn,video_yn,phone_yn
0,shuenmama.223027650182,홍대 회식 맛집,<b>홍대</b> 고기집 데이트 쟁반한상 삼겹살 <b>맛집</b> <b>회식</b>...,https://blog.naver.com/shuenmama/223027650182,shuen,20230225,홍대에서 데이트 하기로 한 주말\n진짜 간만에 홍대\n전에 항상 세우던 공영주차장이...,3094,10,12,0,0,523,3,0,1,1,0
1,rosepink1974.223153722255,홍대 회식 맛집,<b>홍대</b>삼겹살 청년화로 1987 이베리코 연남동고기집 <b>회식맛집</b>,https://blog.naver.com/rosepink1974/223153722255,예쁜 달코미의 단맛 인생,20230712,청년화로1987\n서울 마포구 동교로 219 1층\n청년화로 1987\n홍대입구역 ...,2345,9,9,15,1,418,3,0,1,1,0
2,mou25.223209216526,홍대 회식 맛집,"합정 <b>맛집</b> <b>홍대 회식</b>장소로 딱, 느낌 있는 소고기 고깃집...",https://blog.naver.com/mou25/223209216526,생애 기록장,20230912,"매번 느끼는 거지만,\n회식장소 하나는\n기가 막히게 섭외하는 울 주임님.\n\n얼...",2904,12,12,0,0,554,3,0,1,0,0
3,lulu_l.223118434610,홍대 회식 맛집,<b>홍대</b> <b>맛집</b> 합정 갈비가 부드러운 소고기집 연막탄 <b>회식...,https://blog.naver.com/lulu_l/223118434610,안나의 일상공유,20230602,홍대 소고기 맛집\n연막탄\n\n남자친구가 맛있는 고깃집을 알고\n있다길해 합정 맛...,2810,0,7,9,0,222,3,0,2,0,0
4,ruston_.223161590597,홍대 회식 맛집,"<b>홍대회식</b>, 육즙 폭발하는 소고기 <b>맛집</b> '일편등심 <b>홍대...",https://blog.naver.com/ruston_/223161590597,로빈이 토끼란 사실을 알고있었나?,20230720,안녕하세요. LoLCake입니다.\n\n\n최근 이직을 준비하는 동료의 축하 파티를...,2299,2,2,0,1,573,3,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10484,suitbull.222112955908,영등포 데이트 맛집,"<b>영등포</b> 돈까스 <b>맛집</b>, 타임스퀘어 <b>데이트</b> ::간코::",https://blog.naver.com/suitbull/222112955908,SUITBULL(숯불)의 미식 그리고 풍류,20201011,안녕하세요.\n\n미식을 즐기는\n악식가(樂食家)\nSUITBULL (숯불) 입니다...,1642,0,11,14,0,231,3,0,2,1,1
10485,dusal0652.222413131913,영등포 데이트 맛집,[<b>영등포</b> 문래동] 문래창작촌에서도 유명한 양키통닭,https://blog.naver.com/dusal0652/222413131913,기록하는 습관,20210628,날씨가 좋은 어느 날\n문래에서 일어난 일ㅎㅎㅎㅎㅎ\n\n친구랑 같이 가기로 한 '...,1996,12,12,6,0,171,3,0,2,1,0
10486,shinesparis.221509975993,영등포 데이트 맛집,<b>영등포</b>역 <b>맛집</b> '럭치미' 인도요리 커리 완전맛있어~!,https://blog.naver.com/shinesparis/221509975993,귀여운 서소남매,20190410,인도요리를 좋아하는편이라 신랑이랑 가끔 먹으러가곤한다.\n영등포역에 위치한 인도요리...,2350,0,11,5,0,911,3,0,2,0,0
10487,sleeping_yulmu.223070079287,영등포 데이트 맛집,[서울/여의도] 블링 블링 빛이 나는 맛 좋은 삼겹살! '돈블랑...,https://blog.naver.com/sleeping_yulmu/22307007...,sleeping_day,20230410,예년보다 벚꽃이 일찍 개화한 올해!\n친구들과 꽃놀이는 빼놓을 수가 없죠.\n함께 ...,1745,40,30,0,0,257,3,0,1,0,0
